# Data simulation with Timelag ODE

The goal of this notebook is to describe simulating 1h data using the timelag ODE (Mandel 2016) to run as a data interpolation tool for fuel moisture content. 

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Local modules
from src.utils import read_yml, Dict, plot_styles

ode_params = Dict(read_yml("etc/params_models.yaml", subkey="ode"))

## Adjust Params

- Change timelag constant to 1
- Reduce process variance, something larger like 1e-3
- Reduce data variance, something very small like 1e-6. Don't want to make zero for stability reasons

Checking some combos below

In [ ]:
ode_params

## Run a stretch of data

Constant equilibria of 20\%, no rain

Minute Resolution?

In [ ]:
m0 = 10   # Initial moisture content
Nt = 100  # number of timesteps
h2 = 50 # spinup hours

dat = {
    "fm": np.repeat(10, h2).astype(float),
    "Ed": np.repeat(20, Nt).astype(float),
    "Ew": np.repeat(20, Nt).astype(float),
    "rain": np.repeat(0, Nt).astype(float)
}

In [ ]:
import importlib
import src.moisture_ode 
importlib.reload(src.moisture_ode)


from src.moisture_ode import ODE_FMC

In [ ]:
ode_params.update({
    "T": 10, 
})
mod = ODE_FMC(ode_params)

In [ ]:
x = mod.run_model_single(dat, hours=Nt, h2 = h2)

In [ ]:
ode_params.update({
    "T": 1, # 1h fine fuels
})
mod1 = ODE_FMC(ode_params)
x1 = mod1.run_model_single(dat, hours=Nt, h2 = 5)

In [ ]:
plt.plot(x[0,:], label="T=10")
plt.plot(x1[0,:], label="T=1")
plt.legend()

## Simulate with Realistic Weather

Sine wave Eq and Ed, no rain for now.

In [ ]:
m0 = 10   # Initial moisture content
h2 = 50   # spinup hours



In [ ]:
Nt = 100  # number of timesteps
time = np.linspace(0, Nt, Nt)
amplitude = 10.0
period = 24.0  
offset = 20 
noise = np.random.normal(0, 1, size=len(time))


dat = {
    "Ed": amplitude * np.sin(2 * np.pi * 1/period * time) + offset + noise,
    "Ew": amplitude * np.sin(2 * np.pi * 1/period * time) + offset-5 + noise,
    "rain": np.repeat(0, Nt).astype(float),
    "fm": None
}


plt.plot(dat["Ed"], **plot_styles["Ed"])
plt.plot(dat["Ew"], **plot_styles["Ew"])

In [ ]:
ode_params.update({
    "T": 1, # 1h fine fuels
})
mod1 = ODE_FMC(ode_params)
x1 = mod1.run_model_single(dat, hours=Nt, h2 = 0)

# 10h
ode_params.update({
    "T": 10, 
})
mod10 = ODE_FMC(ode_params)
x10 = mod10.run_model_single(dat, hours=Nt, h2 = 0)

# 100h
ode_params.update({
    "T": 100, 
})
mod100 = ODE_FMC(ode_params)
x100 = mod100.run_model_single(dat, hours=Nt, h2 = 0)

In [ ]:
plt.plot(dat["Ed"], **plot_styles["Ed"])
plt.plot(dat["Ew"], **plot_styles["Ew"])
plt.plot(x1[0,:], '#51F551', label = "1h")
plt.plot(x10[0,:], '#2B822B', label = "10h")
plt.plot(x100[0,:], '#123612', label = "100h")
plt.legend()